# Tutorial Notebook 10: Finetuning for Perturbation Response Prediction

In this tutorial, we will demonstrate how to finetune a Cell2Sentence (C2S) model for perturbation response prediction tasks. This is a critical task in single-cell analysis, where the goal is to predict how a cell's gene expression profile changes in response to a specific perturbation (e.g., a genetic knockout or a drug treatment).

We will treat this as a "translation" task in natural language: translating a cell (in cell sentence format) from its basal (control) state to its perturbed state, conditioned on the perturbation applied.

At a high level, we will:
1. Load a public single-cell perturbation dataset.
2. Write a custom prompt template for perturbation prediction.
3. Subclass the `PromptFormatter` class to create pairs of control and perturbed cells.
4. Finetune a pretrained C2S-Scale model on this new task.
5. Generate a prediction with our new finetuned model to see it in action.

First, let's import the necessary libraries. We'll need standard data science libraries, single-cell analysis tools, and modules from the `cell2sentence` and `transformers` packages.

In [ ]:
# Python built-in libraries
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["WORLD_SIZE"] = "1"

import pickle
import random
from datetime import datetime
from collections import Counter, defaultdict

# Third-party libraries
from datasets import Dataset
import numpy as np
import torch
from transformers import TrainingArguments, AutoModelForCausalLM
from tqdm import tqdm

# Single-cell libraries
import anndata
import scanpy as sc

In [ ]:
# Cell2Sentence imports
import cell2sentence as cs
from cell2sentence.prompt_formatter import get_cell_sentence_str, PromptFormatter

Let's install all the required libraries for this notebook.

In [ ]:
!pip install anndata scanpy datasets transformers tqdm cell2sentence torch

In [ ]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)

# Load Perturbation Data

For this tutorial, you will need a single-cell dataset that includes both control and perturbed cells. The data should be in an `AnnData` object. Crucially, your `.obs` dataframe must contain:
- A column that distinguishes control cells from perturbed cells, e.g., `adata.obs['condition']`
    - Values can be something like 'control' or 'non-targeting' for control cells, and 'perturbed' or the perturbation name for the perturbed cells

For this example, we use a public genetic perturbation dataset of Jurkat cells (Nadig et al., 2025). To use your own dataset, replace `DATA_PATH` with the path to your preprocessed data file.
- Paper: https://www.nature.com/articles/s41588-025-02169-3

<font color='red'>Ensure your data is preprocessed and normalized (e.g., using log1p transformation) before proceeding.</font>

In [ ]:

# Replace this with the actual path to your dataset, if using a custom dataset
DATA_PATH = "/content/drive/MyDrive/Virtual_Cell_Challenge/vcc_data/adata_Training.h5ad"
adata = anndata.read_h5ad(DATA_PATH)
adata

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
adata.obs.head()

In [ ]:
target_gene_counter = Counter(adata.obs['target_gene'])
print(len(target_gene_counter))

In [ ]:
target_gene_counter.most_common(20)

This data contains both control cells (non-targeting) as well as cells under different genetic knockouts.

In [ ]:
adata.var.head()

In [ ]:
adata.X

In [ ]:
# View a few nonzero values
adata.X.data[:10]

In [ ]:
adata.X.max()

The expression is already preprocessed and log1p transformed. Typically, a log1p transform with base=10 would be used for Cell2Sentence training if we wish to convert generated cell sentences back to expression vectors, since log base=10 gives a better linear relationship between log rank and log expression in many single-cell datasets.

For this tutorial, we will use this processed data as is, to train our model to generate target cell sentences.

# Cell2Sentence Conversion

Now, we will convert the gene expression data in our `AnnData` object into cell sentences. This process creates a Hugging Face Arrow dataset, which is used in our LLM training.

In [ ]:
# We'll keep all relevant columns for our new task
adata_obs_cols_to_keep = adata.obs.columns.tolist()
adata_obs_cols_to_keep

In [ ]:
# Create Arrow dataset and vocabulary
arrow_ds, vocabulary = cs.CSData.adata_to_arrow(
    adata=adata,
    random_state=SEED,
    sentence_delimiter=' ',
    label_col_names=adata_obs_cols_to_keep
)
arrow_ds

# Custom Prompt Formatting for Perturbation Prediction

This is the core of our tutorial. For this dataset, we have a single large pool of control cells (labeled 'non-targeting') and multiple groups of perturbed cells, each with a specific `target_gene`. Our goal is to create training pairs where each perturbed cell is matched with a randomly selected control cell. Note that for different perturbation applications, there may be better ways of pairing control and perturbed cells.

We will define a custom prompt structure that frames our task for the LLM. The input will contain the **control cell sentence** and the **perturbation name**. The model's expected output (the response) will be the **perturbed cell sentence**.

First, let's define our prompt templates.

In [ ]:
# The input provides the control cell and the perturbation, asking for the perturbed result.
custom_input_prompt_template = """Given the following cell sentence of {num_genes} expressed genes representing a cell's basal state, predict the cell sentence after applying the perturbation: {perturbation_name}.
Control cell sentence: {control_cell_sentence}.

Perturbed cell sentence:"""

# The answer is simply the target cell sentence.
answer_template = "{perturbed_cell_sentence}."

To apply this template, we need to create pairs of (control cell, perturbed cell) for each perturbation. We'll create a custom `PerturbationPromptFormatter` by subclassing the base `PromptFormatter`.

Our custom `format_hf_ds` function will:
1.  First, iterate through the entire dataset to create a list of all control cell indices.
2.  Simultaneously, it will group the indices of all perturbed cells into a dictionary, with the perturbation name (`target_gene`) as the key.
3.  Then, it will iterate through each perturbation group and, for every perturbed cell, randomly select a control cell from the global pool to form a pair.
4.  Finally, it will format these pairs into the input prompts and expected responses for the model.

In [ ]:
from collections import defaultdict

class PerturbationPromptFormatter(PromptFormatter):
    def __init__(self,
        task_name,
        input_prompt,
        answer_template,
        top_k_genes,
        perturbation_col='target_gene',
        control_label='non-targeting'
    ):
        """
        Initializes the custom prompt formatter.

        Args:
            task_name (str): The name for this task.
            input_prompt (str): The template for the model's input.
            answer_template (str): The template for the model's expected response.
            top_k_genes (int): The number of top genes to include in the cell sentence.
            perturbation_col (str): The column name in the dataset that contains perturbation info.
            control_label (str): The label used to identify control cells in the perturbation_col.
        """
        super().__init__()
        self.task_name = task_name
        self.input_prompt = input_prompt
        self.answer_template = answer_template
        self.top_k_genes = top_k_genes
        self.perturbation_col = perturbation_col
        self.control_label = control_label
        assert top_k_genes > 0, "'top_k_genes' must be an integer > 0"

    def format_hf_ds(self, hf_ds):
        """
        Custom formatting function for perturbation prediction. This function creates pairs of
        (control, perturbed) cells by sampling from a global pool of control cells.
        """
        model_inputs_list = []
        responses_list = []

        # 1. Separate all cells into a global control pool and a dict of perturbed cells
        control_indices = []
        pert_to_indices = defaultdict(list)

        print("Grouping cells by perturbation and identifying global controls...")
        for i, sample in enumerate(hf_ds):
            if sample[self.perturbation_col] == self.control_label:
                control_indices.append(i)
            else:
                pert_to_indices[sample[self.perturbation_col]].append(i)

        assert len(control_indices) > 0, "No control cells found. Cannot create pairs."
        print(f"Found {len(control_indices)} control cells.")
        print(f"Found {len(pert_to_indices)} unique perturbations.")

        # 2. Create prompt-response pairs by iterating through perturbed cells
        print("Creating control-perturbed pairs...")
        for pert_name, perturbed_indices in tqdm(pert_to_indices.items()):
            for perturbed_idx in perturbed_indices:
                # Pair each perturbed cell with a random control cell from the global pool
                control_idx = random.choice(control_indices)

                control_sample = hf_ds[control_idx]
                perturbed_sample = hf_ds[perturbed_idx]

                # Format control cell sentence
                control_sentence, num_genes_str = get_cell_sentence_str(
                    control_sample,
                    num_genes=self.top_k_genes
                )
                # Format perturbed cell sentence
                perturbed_sentence, _ = get_cell_sentence_str(
                    perturbed_sample,
                    num_genes=self.top_k_genes
                )

                # Format the model input string using the perturbation name
                model_input_str = self.input_prompt.format(
                    num_genes=num_genes_str,
                    perturbation_name=pert_name,
                    control_cell_sentence=control_sentence
                )

                # Format the response string
                response_str = self.answer_template.format(
                    perturbed_cell_sentence=perturbed_sentence
                )

                model_inputs_list.append(model_input_str)
                responses_list.append(response_str)

        # Create the final Hugging Face Dataset
        ds_split_dict = {
            "sample_type": [self.task_name] * len(model_inputs_list),
            "model_input": model_inputs_list,
            "response": responses_list,
        }
        ds = Dataset.from_dict(ds_split_dict)
        return ds

Let's instantiate our custom formatter and test it on a small sample of our data to see the result.

In [ ]:
task_name = "perturbation_prediction"
prompt_formatter = PerturbationPromptFormatter(
    task_name=task_name,
    input_prompt=custom_input_prompt_template,
    answer_template=answer_template,
    top_k_genes=200 # Using top 200 genes for this example. For real applications, ideal to use all nonzero expressed genes if possible.
)

In [ ]:
# Format the dataset
formatted_ds = prompt_formatter.format_hf_ds(arrow_ds)
formatted_ds

Note that if you want to do a train/test split of the data, separating out a split of control cells and holdout perturbed cells / entire perturbations can be done before formatting.

In [ ]:
# Inspect a formatted sample
print("--- Formatted Sample ---")
print("#----Model input:----#")
print(formatted_ds[0]["model_input"], "\n")
print("#----Response:----#")
print(formatted_ds[0]["response"])

Now that our custom formatter is ready, we'll wrap our original `arrow_ds` in a `CSData` object. The `finetune` function will use this object and our custom formatter to prepare the data for training.

In [ ]:
# Save directory for Huggingface dataset
c2s_save_dir = "/home/sr2464/scratch/C2S_API_Testing/Data/perturbation_tutorial/perturbation_tutorial"
c2s_save_name = "jurkat_perturbation_c2s"

In [ ]:
csdata = cs.CSData.csdata_from_arrow(
    arrow_dataset=arrow_ds,  # Regular cell sentence dataset put here, finetune() function will repeat the formatting with the prompt formatter
    vocabulary=vocabulary,
    save_dir=c2s_save_dir,
    save_name=c2s_save_name,
    dataset_backend="arrow"
)
print(csdata)

# Load a Pretrained Cell2Sentence Model

We will start with a C2S model that has already been pretrained on a diverse set of single-cell datasets. This provides a strong foundation of biological knowledge. The `C2S-Scale-Pythia-1b-pt` and newer C2S-Scale models are good general-purpose models to start from.

In [ ]:
model_name_or_path = "vandijklab/C2S-Scale-Pythia-1b-pt"
save_dir = "/home/sr2464/scratch/C2S_API_Testing/Cache_Dir/perturbation_tutorial"
save_name = "perturbation_pythia_1B"

In [ ]:
csmodel = cs.CSModel(
    model_name_or_path=model_name_or_path,
    save_dir=save_dir,
    save_name=save_name
)
print(csmodel)

# Finetune for Perturbation Prediction

Now, we'll finetune our model on the perturbation prediction task. We'll define our `TrainingArguments` and then call the `finetune()` method, passing in our `csdata` object and the `PerturbationPromptFormatter` instance we created.

For this tutorial, we'll run for a small number of steps (`max_steps=500`). For a full finetuning run, you would typically train for several epochs.

In [ ]:
datetimestamp = datetime.now().strftime('%Y-%m-%d-%H_%M_%S')
output_dir = os.path.join(csmodel.save_dir, f"{datetimestamp}_finetune_{task_name}")
print(output_dir)

In [ ]:
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [ ]:
train_args = TrainingArguments(
    bf16=True,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=1e-5,
    logging_steps=50,
    lr_scheduler_type="cosine",
    num_train_epochs=1,
    eval_steps=50,
    evaluation_strategy="steps",
    save_steps=100,
    save_strategy="steps",
    output_dir=output_dir,
    max_steps=500  # Shortened for tutorial purposes
)

In [ ]:
csmodel.fine_tune(
    csdata=csdata,
    task=task_name,
    train_args=train_args,
    loss_on_response_only=True, # We only want to calculate loss on the predicted sentence
    top_k_genes=200,  # Use top 200 genes for this example, normally would use full cell sentence (all nonzero expressed genes) if possible
    prompt_formatter=prompt_formatter  # Pass in our custom prompt formatter
)

# Generate Predictions with the Finetuned Model

After finetuning, let's load our new model and use it to predict the response to a perturbation for a cell from our test set.

In [ ]:
final_ckpt_path = os.path.join(output_dir, "checkpoint-500")
final_ckpt_path

In [ ]:
save_dir

In [ ]:
# Load the finetuned model (it's automatically saved to csmodel.model_name_or_path)
finetuned_model = cs.CSModel(
    model_name_or_path=final_ckpt_path, # Path is updated after finetuning
    save_dir=save_dir,
    save_name="perturbation_predictor_finetuned_final"
)

In [ ]:
finetuned_model.save_path

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
final_model = AutoModelForCausalLM.from_pretrained(
    finetuned_model.save_path,
    cache_dir=os.path.join(csmodel.save_dir, ".cache"),
    trust_remote_code=True
).to(device)

In [ ]:
# Load dataset split done in finetune() function, saved to output directory
with open(os.path.join(output_dir, 'data_split_indices_dict.pkl'), 'rb') as f:
    data_split_indices_dict = pickle.load(f)

data_split_indices_dict.keys()

In [ ]:
# Print a few indices of test samples
data_split_indices_dict['test'][:10]

In [ ]:
# Select a few unseen samples
formatted_test_ds = formatted_ds.select(data_split_indices_dict['test'][:10])
formatted_test_ds

In [ ]:
# Select a sample from the test set for inference
sample_idx = 0
inference_prompt = formatted_test_ds[sample_idx]['model_input']
ground_truth_response = formatted_test_ds[sample_idx]['response']

print("--- Inference Prompt ---")
print(inference_prompt)

In [ ]:
# Generate the prediction
predicted_response = finetuned_model.generate_from_prompt(
    model=final_model,
    prompt=inference_prompt,
    max_num_tokens=800 # max number of tokens to generate, ~4 tokens per gene
)

In [ ]:
print("\n--- Ground Truth Perturbed Cell ---")
print(ground_truth_response)
print("\n--- Predicted Perturbed Cell ---")
print(predicted_response)

# Conclusion

In this tutorial, you learned how to finetune a Cell2Sentence model for a custom task: perturbation response prediction. By creating a `PerturbationPromptFormatter`, we were able to structure our data into control-perturbation-response triplets, enabling the model to learn the complex transcriptional changes that occur upon perturbation.

This approach is highly flexible and can be adapted to various experimental designs. The finetuned model can now be used for in-silico experiments, such as virtual screening of genetic perturbations or predicting the effect of new compounds, significantly accelerating the pace of biological discovery.